<a href="https://colab.research.google.com/github/nagasatvika/semantic-similarity/blob/main/sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("paws", "labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

#Jaccard Similarity

In [26]:
def preprocess_text(text):
    text = text.lower()
    return set(text.split())
def Jaccard_Similarity(doc1, doc2):
    intersection = doc1.intersection(doc2)
    union = doc1.union(doc2)
    return float(len(intersection)) / len(union)
for example in dataset['test'].select(range(5)):
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']

    doc1 = preprocess_text(sentence1)
    doc2 = preprocess_text(sentence2)

    similarity = Jaccard_Similarity(doc1, doc2)
    print(f'sentence1:"{sentence1}"\nsentence2:"{sentence2}"\nscore:{similarity}\n')

sentence1:"This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic ."
sentence2:"This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic ."
score:0.8620689655172413

sentence1:"His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America ."
sentence2:"His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri ."
score:0.92

sentence1:"In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan ."
sentence2:"In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan inspected the 

#Using Sentence Transformers

In [19]:
!pip3 install sent2vec

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [20]:
from sent2vec.vectorizer import Vectorizer
from scipy import spatial
vectorizer = Vectorizer(pretrained_weights='distilbert-base-uncased', ensemble_method='average')

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [25]:
def preprocess_text(text):
    text = text.lower()
    return set(text.split())
for example in dataset['test'].select(range(5)):
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']

    doc1 = preprocess_text(sentence1)
    doc2 = preprocess_text(sentence2)

    vectorizer.run([sentence1,sentence2])
    vectors = vectorizer.vectors

    cosine_dist = spatial.distance.cosine(vectors[0],vectors[1])
    print(f'sentence1:"{sentence1}"\nsentence2:"{sentence2}"\nscore:{cosine_dist}\n')


sentence1:"This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic ."
sentence2:"This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic ."
score:0.004304349422454834

sentence1:"His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America ."
sentence2:"His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri ."
score:0.004304349422454834

sentence1:"In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan ."
sentence2:"In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangili

#Using Spacy Similiarity

In [7]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import spacy
nlp = spacy.load('en_core_web_md')

In [24]:
p1 = dataset['test']['sentence1'][:10]
p2 = dataset['test']['sentence2'][:10]

similarities = []

for sentence1,sentence2 in zip(p1,p2):
  W1 = nlp(sentence1)
  W2 = nlp(sentence2)
  Similarity = W1.similarity(W2)
  print(f'sentence1:"{sentence1}"\nsentence2:"{sentence2}" \nscore:{Similarity}\n')

sentence1:"This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic ."
sentence2:"This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic ." 
score:0.9973651350651884

sentence1:"His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America ."
sentence2:"His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri ." 
score:0.997916950603778

sentence1:"In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan ."
sentence2:"In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan